In [54]:
import os
import pydicom
import nibabel as nib
import numpy as np
from tqdm import tqdm  # 진행 상태 표시
import matplotlib.pyplot as plt
import pandas as pd
from operator import itemgetter

In [55]:
def find_dicom_folders(base_path, target_folder_names):
    dicom_folders = []
    failed_folders = []
    #아래 폴더들까지 재귀적 탐색 진행
    def recursive_search(current_path):
        for root, dirs, files in os.walk(current_path):
            for target_folder_name in target_folder_names:
                # 타겟 폴더 탐색
                if target_folder_name in dirs:
                    target_path = os.path.join(root, target_folder_name)
                    #print(f"🔍 [타겟 폴더 발견]: {target_path}")

                    # 타겟 폴더 내부 탐색 (여러 시점 폴더 포함 - target folder 내에 여러 시점의 데이터가 다시 들어가있는 경우작 존재함)
                    for sub_root, sub_dirs, sub_files in os.walk(target_path):
                        for sub_dir in sub_dirs:
                            sub_dir_path = os.path.join(sub_root, sub_dir)
                            dcm_files = [f for f in os.listdir(sub_dir_path) if f.lower().endswith('.dcm')]

                            if dcm_files:  # DICOM 파일 존재
                                dicom_folders.append(sub_dir_path)
                                print(f"[V] [DICOM 발견]: {sub_dir_path}")
                            else:
                                failed_folders.append(sub_dir_path)
                                print(f"[X] [DICOM 없음]: {sub_dir_path}")

    # 최상위 폴더부터 시작
    recursive_search(base_path)

    return dicom_folders, failed_folders


def dicom_to_nifti(dicom_folder, output_path):
    # DICOM 파일 읽기
    dicom_files = [os.path.join(dicom_folder, f) for f in os.listdir(dicom_folder) if f.lower().endswith('.dcm')]
    dicom_files.sort()  # 파일 정렬 - 이거 데이터 받아와서 position 기준으로 해야할지는 volume 출력해봐야 알듯 

    if not dicom_files:
        print(f"[X] [오류] {dicom_folder}에 DICOM 파일이 없습니다.")
        return
    
    try:
        # 3D 볼륨 생성
        slices = [pydicom.dcmread(dcm).pixel_array for dcm in dicom_files]
        volume = np.stack(slices, axis=-1)  # HxWxD 형태로 스택

        # NIfTI 파일 저장
        affine = np.eye(4)  # 기본 affine 행렬
        nifti_img = nib.Nifti1Image(volume, affine)
        nib.save(nifti_img, output_path)
        #print(f"[V] [NIfTI 저장 완료]: {output_path}")
    except Exception as e:
        pass
        #print(f"[X] [오류 발생]: {dicom_folder} - {e}")



# 1. 3D Volume화를 시킥 경우 -> 하나의 폴더로 싹 담고, 이후에 Dataframe 따로 이용해서 MMSE Score에 대한 정보 담기
# 2. Multi-View 이용할 경우 : 얘네는 적으니까 폴더 구조 그대로 유지해서 진행한다.
def convert_adni_dicom_to_nifti(base_path, target_folder_names, output_base_path):
    # Step 1: 대상 DICOM 폴더 찾기
    print("탐색시작")
    dicom_folders, failed_folders = find_dicom_folders(base_path, target_folder_names)
    print(f"[V] 총 {len(dicom_folders)}개의 DICOM 폴더를 찾았습니다.")
    print(f"[X] 총 {len(failed_folders)}개의 폴더에서 DICOM 파일을 찾을 수 없습니다.")
    # 디버깅용 코드입니다.
    print(failed_folders)
    
    # Patient, MRI 정보를 저장할 리스트
    records = []

    for dicom_folder in tqdm(dicom_folders, desc="DICOM → NIfTI 변환 진행"):
        # relative_path와 관련 정보 추출
        relative_path = os.path.relpath(dicom_folder, base_path)
        rp_arr = relative_path.split(os.sep)  # os.sep으로 플랫폼에 맞게 경로 분리
        patient_number = rp_arr[0]
        mri_number = rp_arr[-1]
        visdate=rp_arr[1]
        
        print('patient number:', patient_number)
        print('MRI number :', mri_number)
        
        # 정보 저장
        records.append({"Patient Number": patient_number, "VISDATE":visdate,"MRI Number": mri_number})
        
        # 출력 폴더 설정
        output_folder = os.path.join(output_base_path)  # 폴더 구조 없이 저장
        os.makedirs(output_folder, exist_ok=True)

        # 출력 파일명 설정
        output_file = os.path.join(output_folder, f"{patient_number},{visdate },{mri_number}.nii.gz")
        print('Output file:', output_file)
        
        # DICOM → NIfTI 변환
        dicom_to_nifti(dicom_folder, output_file)

    # 실패한 폴더 출력
    if failed_folders:
        print("\n[X] [조건을 만족하지 못한 폴더]:")
        for folder in failed_folders:
            print(f" - {folder}")
    
    # DataFrame 생성 및 저장
    if records:
        df = pd.DataFrame(records)
        csv_path = os.path.join(output_base_path, "patient_mri_mapping.csv")
        df.to_csv(csv_path, index=False, encoding='utf-8-sig')
        print(f"\n[V] Patient-MRI 매핑 데이터가 {csv_path}에 저장되었습니다.")

In [60]:
 # 최상위 ADNI 폴더 경로 설정
base_path = "F:\\MRI 파일\\ADNI3"

# 찾고자 하는 타겟 폴더 이름 리스트
target_folder_names = [
"Accelerated_Sagittal_MPRAGE",
"Sagittal_3D_Accelerated_MPRAGE",
"Accelerated_Sag_IR-FSPGR",
"Accelerated_Sagittal_IR-FSPGR",
"Sag_Accel_IR-FSPGR",
"Sagittal_3D_Accelerated_0_angle_MPRAGE",
"Accelerated_Sagittal_MPRAGE_Phase_A-P",
"Accelerated_Sagittal_MPRAGE__MSV21",
"Accelerated_Sagittal_MPRAGE__MSV21_",
"Accelerated_Sagittal_MPRAGE__MSV21_RPT",
"Accelerated_Sagittal_IR-FSPGR__MSV21",
"Accelerated_Sagittal_IR-FSPGR__MSV21_",
"Accelerated_Sagittal_IR-FSPGR__MSV21__RPT",
"Accelerated_Sagittal_IR-FSPGR__MSV22",
"Accelerated_Sagittal_IR-FSPGR__MSV22_",
"Sagittal_3D_Accelerated_MPRAGE_MSV21_"
]

# NIfTI 파일 저장 경로
output_base_path = "./niis/ADNI3/USB1"

# 변환 함수 호출
convert_adni_dicom_to_nifti(base_path, target_folder_names, output_base_path)

탐색시작
[V] [DICOM 발견]: F:\MRI 파일\ADNI3\002_S_0413\2017-06-21\Accelerated_Sagittal_MPRAGE\I863056
[V] [DICOM 발견]: F:\MRI 파일\ADNI3\002_S_0413\2019-08-27\Accelerated_Sagittal_MPRAGE\I1221051
[V] [DICOM 발견]: F:\MRI 파일\ADNI3\002_S_1155\2017-04-24\Accelerated_Sagittal_MPRAGE\I843510
[V] [DICOM 발견]: F:\MRI 파일\ADNI3\002_S_1155\2018-05-08\Accelerated_Sagittal_MPRAGE\I995496
[V] [DICOM 발견]: F:\MRI 파일\ADNI3\002_S_1155\2019-08-16\Accelerated_Sagittal_MPRAGE\I1270004
[V] [DICOM 발견]: F:\MRI 파일\ADNI3\002_S_1261\2017-03-15\Accelerated_Sagittal_MPRAGE\I831065
[V] [DICOM 발견]: F:\MRI 파일\ADNI3\002_S_1261\2018-04-24\Accelerated_Sagittal_MPRAGE\I989320
[V] [DICOM 발견]: F:\MRI 파일\ADNI3\002_S_1261\2019-05-01\Accelerated_Sagittal_MPRAGE\I1270020
[V] [DICOM 발견]: F:\MRI 파일\ADNI3\002_S_1280\2017-03-13\Accelerated_Sagittal_MPRAGE\I829296
[V] [DICOM 발견]: F:\MRI 파일\ADNI3\002_S_1280\2019-03-06\Accelerated_Sagittal_MPRAGE\I1140407
[V] [DICOM 발견]: F:\MRI 파일\ADNI3\002_S_4213\2017-08-14\Accelerated_Sagittal_MPRAGE\I888008
[

DICOM → NIfTI 변환 진행:   0%|          | 0/2142 [00:00<?, ?it/s]

patient number: 002_S_0413
MRI number : I863056
Output file: ./niis/ADNI3/USB1\002_S_0413,2017-06-21,I863056.nii.gz


DICOM → NIfTI 변환 진행:   0%|          | 1/2142 [00:03<2:11:27,  3.68s/it]

patient number: 002_S_0413
MRI number : I1221051
Output file: ./niis/ADNI3/USB1\002_S_0413,2019-08-27,I1221051.nii.gz


...

patient number: 941_S_6854
MRI number : I1547737
Output file: ./niis/ADNI3/USB1\941_S_6854,2022-02-22,I1547737.nii.gz


DICOM → NIfTI 변환 진행: 100%|█████████▉| 2141/2142 [2:05:52<00:03,  3.66s/it]

patient number: 941_S_7106
MRI number : I1619403
Output file: ./niis/ADNI3/USB1\941_S_7106,2022-09-09,I1619403.nii.gz


DICOM → NIfTI 변환 진행: 100%|██████████| 2142/2142 [2:05:56<00:00,  3.53s/it]


[X] [조건을 만족하지 못한 폴더]:
 - F:\MRI 파일\ADNI3\024_S_2239\2021-10-20\Accelerated_Sagittal_MPRAGE\2021-10-20_09_27_54.0
 - F:\MRI 파일\ADNI3\024_S_2239\2021-10-20\Accelerated_Sagittal_MPRAGE\2021-10-20_10_10_15.0
 - F:\MRI 파일\ADNI3\037_S_4410\2017-03-01\Accelerated_Sagittal_MPRAGE\2017-03-01_10_05_07.0
 - F:\MRI 파일\ADNI3\137_S_4520\2020-11-12\Accelerated_Sagittal_MPRAGE\2020-11-12_08_02_52.0
 - F:\MRI 파일\ADNI3\168_S_6371\2023-07-27\Accelerated_Sagittal_MPRAGE\2023-07-27_13_22_12.0
 - F:\MRI 파일\ADNI3\168_S_6371\2023-07-27\Accelerated_Sagittal_MPRAGE\2023-07-27_13_38_32.0
 - F:\MRI 파일\ADNI3\941_S_6570\2018-08-31\Accelerated_Sagittal_MPRAGE\2018-08-31_10_05_11.0

[V] Patient-MRI 매핑 데이터가 ./niis/ADNI3/USB1\patient_mri_mapping.csv에 저장되었습니다.


In [57]:
nii_file_path =r"C:\Users\user\Desktop\Projects\NRF\MRI_Alzheimer\niis\003_S_6954\2021-05-26\Accelerated_Sagittal_MPRAGE\I1448643.nii.gz"  # 확인할 NIfTI 파일 경로
nifti_data = nib.load(nii_file_path)

# 데이터 정보 출력
print("파일 경로:", nii_file_path)
print("데이터 타입:", type(nifti_data))
print("이미지 모양:", nifti_data.shape)
print("Affine 행렬:\n", nifti_data.affine)  # 공간 좌표 변환 정보
print("헤더 정보:\n", nifti_data.header)   # 헤더 정보

# NIfTI 데이터 배열 추출
image_data = nifti_data.get_fdata()  # 3D 또는 4D 데이터 배열
print("데이터 배열 모양:", image_data.shape)

# 3D 데이터 시각화 (중앙 슬라이스)
mid_slice = image_data[:, :, image_data.shape[2] // 2]  # Z축 중간 슬라이스
plt.imshow(mid_slice.T, cmap="gray", origin="lower")
plt.title("NIfTI 중앙 슬라이스")
plt.axis("off")
plt.show()

FileNotFoundError: No such file or no access: 'C:/Users/user/Desktop/Projects/NRF/MRI_Alzheimer/niis/003_S_6954/2021-05-26/Accelerated_Sagittal_MPRAGE/I1448643.nii.gz'